<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/Testing_a_code_for_igbobert4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
#!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt
!wget -c https://github.com/chiamaka249/IgboNER/blob/main/for_test.txt
# !wget -c https://github.com/chiamaka249/IgboNER/blob/main/config.json

--2021-12-10 13:46:22--  https://github.com/chiamaka249/IgboNER/blob/main/for_test.txt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘for_test.txt’

for_test.txt            [ <=>                ] 175.01K  --.-KB/s    in 0.02s   

2021-12-10 13:46:22 (7.51 MB/s) - ‘for_test.txt’ saved [179210]



In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-882zs4sb
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-882zs4sb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 9.2 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 3.3 MB 61.2 MB/s 
     |████████████████████████████████| 61 kB 621 kB/s 
  Created wheel for transformers: filename=transformers-4.14.0.dev0-py3-none-any.whl size=3320665 sha256=cd4db7bbc4c351dd38ce2bb391fc44e9108c5a056a1bac37f610fd7de55221ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-_jyqoi8c/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

In [ ]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=2_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 228 ms, sys: 80.3 ms, total: 309 ms
Wall time: 103 ms


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')




Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/drive/My Drive/'
/content


In [ ]:
!mkdir igbo_bert44
tokenizer.save_model("igbo_bert44")

['igbo_bert44/vocab.json', 'igbo_bert44/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert44/vocab.json",
    "./igbo_bert44/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.").tokens

['<s>',
 'S',
 'im',
 'on',
 'e',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥',
 'n',
 'ya',
 'h',
 'á»¥',
 'Ġg',
 'u',
 'á»į',
 'Ġe',
 'gw',
 'u',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġt',
 'aa',
 '.',
 '</s>']

In [ ]:
# Check that we have a GPU
!nvidia-smi

Wed Dec  8 12:00:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=2_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
!wget -c https://github.com/chiamaka249/IgboNER/blob/main/config.json
import shutil
shutil.copy('/content/config.json', '/content/igbo_bert44') 

--2021-12-10 13:48:28--  https://github.com/chiamaka249/IgboNER/blob/main/config.json
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘config.json’

config.json             [ <=>                ] 164.94K  --.-KB/s    in 0.02s   

2021-12-10 13:48:29 (6.63 MB/s) - ‘config.json’ saved [168898]



'/content/igbo_bert44/config.json'

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert44", max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

45054416

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/for_test.txt",
    block_size = 128
)

CPU times: user 548 ms, sys: 366 ms, total: 915 ms
Wall time: 515 ms


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert44",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 1319
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 21
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 8.1 s, sys: 18.4 ms, total: 8.12 s
Wall time: 8.22 s


TrainOutput(global_step=21, training_loss=6.607648577008929, metrics={'train_runtime': 8.196, 'train_samples_per_second': 160.931, 'train_steps_per_second': 2.562, 'total_flos': 43683148664832.0, 'train_loss': 6.607648577008929, 'epoch': 1.0})

In [ ]:
trainer.save_model("./content/gdrive/MyDrive/2017-07-07 row/igbo_bert44")

Saving model checkpoint to ./content/gdrive/MyDrive/2017-07-07 row/igbo_bert44
Configuration saved in ./content/gdrive/MyDrive/2017-07-07 row/igbo_bert44/config.json
Model weights saved in ./content/gdrive/MyDrive/2017-07-07 row/igbo_bert44/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert4",
    tokenizer="./igbo_bert4"
)

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

In [ ]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.006366073153913021,
  'sequence': 'Abụ m Maazị A.',
  'token': 347,
  'token_str': ' A'},
 {'score': 0.0061102816835045815,
  'sequence': 'Abụ m Maazị M.',
  'token': 395,
  'token_str': ' M'},
 {'score': 0.005945019423961639,
  'sequence': 'Abụ m Maazị O.',
  'token': 377,
  'token_str': ' O'},
 {'score': 0.004409363958984613,
  'sequence': 'Abụ m Maazị Obi.',
  'token': 926,
  'token_str': ' Obi'},
 {'score': 0.003987359814345837,
  'sequence': 'Abụ m Maazị Chineke.',
  'token': 403,
  'token_str': ' Chineke'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.12052495032548904,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.07421518117189407,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 732,
  'token_str': ' nwunye'},
 {'score': 0.04338353872299194,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 620,
  'token_str': ' nwaanyị'},
 {'score': 0.03189236298203468,
  'sequence': 'Nwaanyị na- ji na akara.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.0238909013569355,
  'sequence': 'Nwaanyị na nne ji na akara.',
  'token': 902,
  'token_str': ' nne'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.35245102643966675,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 718,
  'token_str': ' ọjọọ'},
 {'score': 0.17003658413887024,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 498,
  'token_str': ' ọma'},
 {'score': 0.08485106378793716,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 430,
  'token_str': ' niile'},
 {'score': 0.047518376260995865,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.01763184554874897,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ùgwù.',
  'token': 1300,
  'token_str': ' ùgwù'}]

In [ ]:
fill_mask("O riri <mask>.")

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.3398936986923218,
  'sequence': 'O riri ya.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.0448780357837677,
  'sequence': 'O riri mmiri.',
  'token': 703,
  'token_str': ' mmiri'},
 {'score': 0.03440803661942482,
  'sequence': 'O riri ha.',
  'token': 297,
  'token_str': ' ha'},
 {'score': 0.03390880301594734,
  'sequence': 'O riri ihe.',
  'token': 302,
  'token_str': ' ihe'},
 {'score': 0.012751125730574131,
  'sequence': 'O riri afọ.',
  'token': 485,
  'token_str': ' afọ'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.07749678939580917,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.026143409311771393,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 302,
  'token_str': ' ihe'},
 {'score': 0.026121513918042183,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 873,
  'token_str': ' nri'},
 {'score': 0.014630773104727268,
  'sequence': 'Nwaanyị na eri di na akara.',
  'token': 686,
  'token_str': ' di'},
 {'score': 0.013305693864822388,
  'sequence': 'Nwaanyị na eri mmiri na akara.',
  'token': 703,
  'token_str': ' mmiri'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.03819141909480095,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.02734088897705078,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1516,
  'token_str': ' Nwanna'},
 {'score': 0.02290385030210018,
  'sequence': 'Ọ bụ nwunyea ka a na- arịa.',
  'token': 732,
  'token_str': ' nwunye'},
 {'score': 0.01657039113342762,
  'sequence': 'Ọ bụ nwaa ka a na- arịa.',
  'token': 420,
  'token_str': ' nwa'},
 {'score': 0.011486751958727837,
  'sequence': 'Ọ bụ naanịa ka a na- arịa.',
  'token': 790,
  'token_str': ' naanị'}]

In [ ]:
import shutil

In [ ]:
shutil.make_archive("/content/igbo_bert4", 'zip', "igbo_bert4")

'/content/igbo_bert4.zip'

In [ ]:
from google.colab import files
files.download("/content/igbo_bert4.zip")

FileNotFoundError: ignored